In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import json
from collections import defaultdict
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load and group your data
with open("/content/respone_new.json", "r") as f:
    raw_data = json.load(f)

# Group outputs by identical input
grouped_data = defaultdict(list)
for item in raw_data:
    grouped_data[item["input"]].append(item["output"])

# Flatten into input-output pairs (one per line, preserving duplicates)
final_data = []
for inp, outs in grouped_data.items():
    for out in outs:
        final_data.append({"input": inp, "output": out})

# Add prefix to each input (recommended for T5)
prefix = "generate response: "
input_texts = [prefix + item["input"] for item in final_data]
target_texts = [item["output"] for item in final_data]

# Create a Hugging Face dataset
dataset = Dataset.from_dict({
    "input_text": input_texts,
    "target_text": target_texts
})

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenization function
def preprocess(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=64, return_tensors="pt")

    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input_text", "target_text"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./t5_chatbot",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    learning_rate=3e-4,
    weight_decay=0.01
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# Start training
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./t5_chatbot")
tokenizer.save_pretrained("./t5_chatbot")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: afzalhakkim666 (afzalhakkim666-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


Step,Training Loss
10,3.584600
20,1.620800
30,1.393900
40,1.176300
50,1.161000
60,0.992200
70,0.863300
80,0.852900
90,0.848600
100,0.810700


('./t5_chatbot/tokenizer_config.json',
 './t5_chatbot/special_tokens_map.json',
 './t5_chatbot/spiece.model',
 './t5_chatbot/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./t5_chatbot")
tokenizer = T5Tokenizer.from_pretrained("./t5_chatbot")

# Function to get response
def generate_response(user_input):
    input_text = f"generate response: {user_input}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=128)

    output_ids = model.generate(
    input_ids,
    max_length=30,
    min_length=20,
    num_beams=4,
    early_stopping=True,
    repetition_penalty=1.6,
    do_sample=True,  # Enable sampling for temperature to work
    temperature=0.8,
    no_repeat_ngram_size=1
)


    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return response

test_inputs = [
    "intent: course_info | entity: robotics",
    "intent: course_info | entity: testing",
    "intent: course_info | entity: datascience",
    "intent: course_info | entity: django",
    "intent: course_info | entity: java",
    "intent: course_info | entity: mean",
    "intent: course_info | entity: flutter",
    "intent: course_info | entity: aspnet",

    "intent: fees_inquiry | entity: robotics",
    "intent: fees_inquiry | entity: testing",
    "intent: fees_inquiry | entity: datascience",
    "intent: fees_inquiry | entity: django",
    "intent: fees_inquiry | entity: java",
    "intent: fees_inquiry | entity: mean",
    "intent: fees_inquiry | entity: flutter",
    "intent: fees_inquiry | entity: aspnet",

    "intent: batch_timing | entity: robotics",
    "intent: batch_timing | entity: testing",
    "intent: batch_timing | entity: datascience",
    "intent: batch_timing | entity: django",
    "intent: batch_timing | entity: java",
    "intent: batch_timing | entity: mean",
    "intent: batch_timing | entity: flutter",
    "intent: batch_timing | entity: aspnet",

    "intent: admission_process | entity: robotics",
    "intent: admission_process | entity: testing",
    "intent: admission_process | entity: datascience",
    "intent: admission_process | entity: django",
    "intent: admission_process | entity: java",
    "intent: admission_process | entity: mean",
    "intent: admission_process | entity: flutter",
    "intent: admission_process | entity: aspnet",

    "intent: course_duration | entity: robotics",
    "intent: course_duration | entity: testing",
    "intent: course_duration | entity: datascience",
    "intent: course_duration | entity: django",
    "intent: course_duration | entity: java",
    "intent: course_duration | entity: mean",
    "intent: course_duration | entity: flutter",
    "intent: course_duration | entity: aspnet",

    "intent: learning_mode | entity: robotics",
    "intent: learning_mode | entity: testing",
    "intent: learning_mode | entity: datascience",
    "intent: learning_mode | entity: django",
    "intent: learning_mode | entity: java",
    "intent: learning_mode | entity: mean",
    "intent: learning_mode | entity: flutter",
    "intent: learning_mode | entity: aspnet",

    "intent: contact_info | entity: robotics",
    "intent: contact_info | entity: testing",
    "intent: contact_info | entity: datascience",
    "intent: contact_info | entity: django",
    "intent: contact_info | entity: java",
    "intent: contact_info | entity: mean",
    "intent: contact_info | entity: flutter",
    "intent: contact_info | entity: aspnet",

    "intent: placement_support | entity: robotics",
    "intent: placement_support | entity: testing",
    "intent: placement_support | entity: datascience",
    "intent: placement_support | entity: django",
    "intent: placement_support | entity: java",
    "intent: placement_support | entity: mean",
    "intent: placement_support | entity: flutter",
    "intent: placement_support | entity: aspnet"
]


# Run test
for i, user_input in enumerate(test_inputs):
    print(f"\n🧠 Test {i+1}")
    print("User input:", user_input)
    print("Bot response:", generate_response(user_input))



🧠 Test 1
User input: intent: course_info | entity: robotics
Bot response: The Robotics course covers a wide range of topics, including technical development and project management.

🧠 Test 2
User input: intent: course_info | entity: testing
Bot response: You’ll learn a wide range of tools and techniques to help you navigate the web.

🧠 Test 3
User input: intent: course_info | entity: datascience
Bot response: The Data Science course covers data science, analytics and cloud computing. You’ll also learn how to build a database with the latest technologies in

🧠 Test 4
User input: intent: course_info | entity: django
Bot response: The Django course covers a variety of topics including web development, scripting and project management.

🧠 Test 5
User input: intent: course_info | entity: java
Bot response: You’ll learn how to build a database with the latest technologies and tools for creating real-time applications.

🧠 Test 6
User input: intent: course_info | entity: mean
Bot response: Yo

In [ ]:
!zip -r t5_chatbot.zip t5_chatbot


  adding: t5_chatbot/ (stored 0%)
  adding: t5_chatbot/generation_config.json (deflated 29%)
  adding: t5_chatbot/checkpoint-100/ (stored 0%)
  adding: t5_chatbot/checkpoint-100/generation_config.json (deflated 29%)
  adding: t5_chatbot/checkpoint-100/scheduler.pt (deflated 56%)
  adding: t5_chatbot/checkpoint-100/model.safetensors (deflated 9%)
  adding: t5_chatbot/checkpoint-100/training_args.bin (deflated 51%)
  adding: t5_chatbot/checkpoint-100/config.json (deflated 62%)
  adding: t5_chatbot/checkpoint-100/rng_state.pth (deflated 24%)
  adding: t5_chatbot/checkpoint-100/optimizer.pt (deflated 8%)
  adding: t5_chatbot/checkpoint-100/trainer_state.json (deflated 70%)
  adding: t5_chatbot/model.safetensors (deflated 9%)
  adding: t5_chatbot/checkpoint-200/ (stored 0%)
  adding: t5_chatbot/checkpoint-200/generation_config.json (deflated 29%)
  adding: t5_chatbot/checkpoint-200/scheduler.pt (deflated 56%)
  adding: t5_chatbot/checkpoint-200/model.safetensors (deflated 9%)
  adding: t5_c

In [ ]:
!du -sh t5_chatbot
!du -sh t5_chatbot.zip


1.6G	t5_chatbot
1.5G	t5_chatbot.zip


In [ ]:
!rm -rf t5_chatbot/checkpoint-100
!rm t5_chatbot/checkpoint-200/optimizer.pt
!rm t5_chatbot/checkpoint-200/scheduler.pt
!rm t5_chatbot/checkpoint-200/trainer_state.json
!rm t5_chatbot/checkpoint-200/rng_state.pth


In [ ]:
!zip -r t5_chatbot_trimmed.zip t5_chatbot


  adding: t5_chatbot/ (stored 0%)
  adding: t5_chatbot/generation_config.json (deflated 29%)
  adding: t5_chatbot/model.safetensors (deflated 9%)
  adding: t5_chatbot/checkpoint-200/ (stored 0%)
  adding: t5_chatbot/checkpoint-200/generation_config.json (deflated 29%)
  adding: t5_chatbot/checkpoint-200/model.safetensors (deflated 9%)
  adding: t5_chatbot/checkpoint-200/training_args.bin (deflated 51%)
  adding: t5_chatbot/checkpoint-200/config.json (deflated 62%)
  adding: t5_chatbot/tokenizer_config.json (deflated 94%)
  adding: t5_chatbot/added_tokens.json (deflated 83%)
  adding: t5_chatbot/config.json (deflated 62%)
  adding: t5_chatbot/special_tokens_map.json (deflated 85%)
  adding: t5_chatbot/spiece.model (deflated 48%)


In [ ]:
!du -sh t5_chatbot_trimmed.zip


422M	t5_chatbot_trimmed.zip


In [ ]:
from google.colab import files
files.download("t5_chatbot_trimmed.zip")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>